# Geolocalización con MultiProcessing + IGR Palabras

En esta notebook haremos un intento de geolocalización con los textos de los usuarios...

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_json("../data/geoloc/users_train.json")
df_test = pd.read_json("../data/geoloc/users_test.json")

Hagamos lo siguiente:

- Entrenemos con unigramas una regresión logística para 
- Luego probemos con los regionalismos

Primero, partamos en train, test

In [2]:
df_train.groupby("provincia").count()

,text
provincia,
buenosaires,337
catamarca,341
chaco,331
chubut,328
cordoba,317
corrientes,345
entrerios,338
formosa,286
jujuy,339


## Palabras precalculadas

Carguemos antes las palabras que sabemos que ocurren una cantidad razonable de veces

In [3]:
%%time
from contrastes.processing import build_dataframe_from_users
from contrastes.processing import preprocess_raw_df


#word_df = build_dataframe_from_users(row for index, row in df_train.iterrows())

word_df = pd.read_csv("train_word_df_filtered.csv", index_col=0)
word_df = preprocess_raw_df(word_df, filter_words=(10, 2))

CPU times: user 1.45 s, sys: 220 ms, total: 1.67 s
Wall time: 1.67 s


/home/jmperez/projects/contrastes/contrastes/processing.py:185: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_palabras = cant_palabras
/home/jmperez/projects/contrastes/contrastes/processing.py:186: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_personas = cant_personas


La reg. logística será un softmax, así que elijo `multi_class='multinomial'`

In [4]:
from contrastes.igr import igr

word_df["igr"] = igr(word_df, word_df.columnas_palabras)

In [5]:
word_df.sort_values("igr", ascending=False, inplace=True)

word_df[:30][word_df.columnas_palabras]

,buenosaires_ocurrencias,catamarca_ocurrencias,chaco_ocurrencias,chubut_ocurrencias,cordoba_ocurrencias,corrientes_ocurrencias,entrerios_ocurrencias,formosa_ocurrencias,jujuy_ocurrencias,lapampa_ocurrencias,...,neuquen_ocurrencias,rionegro_ocurrencias,salta_ocurrencias,sanjuan_ocurrencias,sanluis_ocurrencias,santacruz_ocurrencias,santafe_ocurrencias,santiago_ocurrencias,tierradelfuego_ocurrencias,tucuman_ocurrencias
rioja,21.0,219.0,3.0,18.0,23.0,21.0,6.0,2.0,27.0,9.0,...,4.0,7.0,57.0,88.0,3.0,10.0,45.0,11.0,4.0,44.0
logroño,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
hoa,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3275.0,...,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0
ushuaia,19.0,2.0,3.0,16.0,3.0,2.0,3.0,4.0,6.0,7.0,...,7.0,13.0,72.0,8.0,4.0,35.0,6.0,14.0,5640.0,2.0
chepes,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
tiemposur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,883.0,0.0,0.0,0.0,0.0
palpala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,591.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
beder,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
quitilipi,0.0,0.0,1043.0,0.0,0.0,10.0,0.0,3.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oberá,0.0,1.0,1.0,3.0,0.0,2.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0


Veamos qué performance tiene usando 1000, 2000, 3000, y así...

In [6]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from contrastes.text import tokenize

liw_vectorizer = CountVectorizer(
    tokenizer=tokenize,
    vocabulary=word_df.index)

X_train = liw_vectorizer.fit_transform(df_train["text"])
print("Vectorizing")
X_test = liw_vectorizer.transform(df_test["text"])

Vectorizing
CPU times: user 10min 22s, sys: 1.34 s, total: 10min 23s
Wall time: 10min 24s


Ya las tenemos vectorizadas en el orden esperado!

In [7]:
from sklearn.preprocessing import LabelEncoder

province_encoder = LabelEncoder()

province_encoder.fit(df_train["provincia"].values)

y_train = province_encoder.transform(df_train["provincia"].values)
y_test = province_encoder.transform(df_test["provincia"].values)

In [8]:
%%time
from contrastes.classifiers import fit_classifiers

num_words_to_fit = list(range(250, 5000, 250)) + list(range(5000, 20000, 500))

ret = fit_classifiers(X_train, y_train, X_test, y_test, 
                      province_encoder=province_encoder,
                      range_num_words=num_words_to_fit, num_jobs=8)

Entrenando con 750 palabras
Entrenando con 3750 palabras
Entrenando con 1250 palabras
Entrenando con 1750 palabras
Entrenando con 2750 palabras
Entrenando con 2250 palabras
Entrenando con 3250 palabras
Entrenando con 250 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


250   palabras ----> accuracy 42.12 mean distance 592.5516
Entrenando con 500 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


1250  palabras ----> accuracy 60.24 mean distance 401.956
Entrenando con 1500 palabras


...

3750  palabras ----> accuracy 68.12 mean distance 245.236
Entrenando con 4000 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


750   palabras ----> accuracy 56.80 mean distance 436.3248
Entrenando con 1000 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


3250  palabras ----> accuracy 66.92 mean distance 255.4844
Entrenando con 3500 palabras
500   palabras ----> accuracy 51.20 mean distance 502.8532
Entrenando con 4250 palabras
1500  palabras ----> accuracy 62.20 mean distance 377.0852
Entrenando con 4750 palabras
2500  palabras ----> accuracy 65.92 mean distance 260.6856
Entrenando con 5500 palabras
2000  palabras ----> accuracy 63.88 mean distance 344.4592
Entrenando con 6500 palabras
1000  palabras ----> accuracy 59.04 mean distance 411.2016
Entrenando con 7500 palabras
3000  palabras ----> accuracy 66.76 mean distance 253.7084
Entrenando con 8500 palabras
3500  palabras ----> accuracy 67.48 mean distance 251.1044
Entrenando con 9500 palabras
4000  palabras ----> accuracy 68.88 mean distance 240.814
Entrenando con 10500 palabras
4250  palabras ----> accuracy 69.68 mean distance 237.0436
Entrenando con 4500 palabras
4750  palabras ----> accuracy 69.60 mean distance 240.2368
Entrenando con 5000 palabras
5500  palabras ----> accuracy 66

In [10]:
for r in ret:
    num_words = r["num_words"]
    acc = r["accuracy"]
    md = r["mean_distance"]
    print("{:<5} palabras ----> accuracy {:.2f} mean distance {}".format(
        num_words, acc*100, md
    ))

250   palabras ----> accuracy 42.12 mean distance 592.5516
500   palabras ----> accuracy 51.20 mean distance 502.8532
750   palabras ----> accuracy 56.80 mean distance 436.3248
1000  palabras ----> accuracy 59.04 mean distance 411.2016
1250  palabras ----> accuracy 60.24 mean distance 401.956
1500  palabras ----> accuracy 62.20 mean distance 377.0852
1750  palabras ----> accuracy 63.16 mean distance 351.1988
2000  palabras ----> accuracy 63.88 mean distance 344.4592
2250  palabras ----> accuracy 63.92 mean distance 335.9632
2500  palabras ----> accuracy 65.92 mean distance 260.6856
2750  palabras ----> accuracy 66.56 mean distance 252.6876
3000  palabras ----> accuracy 66.76 mean distance 253.7084
3250  palabras ----> accuracy 66.92 mean distance 255.4844
3500  palabras ----> accuracy 67.48 mean distance 251.1044
3750  palabras ----> accuracy 68.12 mean distance 245.236
4000  palabras ----> accuracy 68.88 mean distance 240.814
4250  palabras ----> accuracy 69.68 mean distance 237.0436


In [11]:
import pickle

pickle.dump(ret, open("res_igr_palabras.pkl", "wb"))

In [12]:
new_ret = pickle.load(open("res_igr_palabras.pkl", "rb"))

clf = new_ret[1]["clf"]

clf.coef_.shape

(23, 500)